In [1]:
class Node:
    #Initilise the node
    def __init__(self,puzzle,tmp=0):
        self.children=list()
        self.parent=None
        self.puzzle=list()
        self.x=0
        self.setPuzzle(puzzle)
        self.tmp=tmp
        self.next=None
        #Children: contain nodes to children
        #Parent points to the parent of current node
        #Puzzle contain state
        #X contain location of blank
        #tmp contain cost to node from source
        #next used in hashing in case 2 nodes have same hashing
    #Inserts state into self state from given state
    def setPuzzle(self,puzzle):
        for i in range(len(puzzle)):
            self.puzzle.append(puzzle[i])
    #Check if current state is goal state
    def goalTest(self):
        return self.puzzle==goal
    #Perform right move
    def moveToRight(self,p,i):
        #p is state
        #i is current blank place
        if i%3<2:#Check if not right most
            pc=[]
            self.copyPuzzle(pc,p)
            
            temp=pc[i+1]
            pc[i+1]=pc[i]
            pc[i]=temp
            
            child=Node(pc,self.tmp+1)
            self.children.append(child)
            child.parent=self
    #Perform left move
    def moveToLeft(self,p,i):
        if i%3>0:
            pc=[]
            self.copyPuzzle(pc,p)
            
            temp=pc[i-1]
            pc[i-1]=pc[i]
            pc[i]=temp
            
            child=Node(pc,self.tmp+1)
            self.children.append(child)
            child.parent=self
    #Perform up move   
    def moveToUp(self,p,i):
        if i-3>=0:
            pc=[]
            self.copyPuzzle(pc,p)  
            
            temp=pc[i-3]
            pc[i-3]=pc[i]
            pc[i]=temp
            
            child=Node(pc,self.tmp+1)
            self.children.append(child)
            child.parent=self
    #Perform down move
    def moveToDown(self,p,i):
        if i+3<len(self.puzzle):
            pc=[]
            self.copyPuzzle(pc,p)  
            
            temp=pc[i+3]
            pc[i+3]=pc[i]
            pc[i]=temp
            
            child=Node(pc,self.tmp+1)
            self.children.append(child)
            child.parent=self
    #Print puzzle
    def printPuzzle(self):
        for i in range(len(self.puzzle)):
            if i%3==0:
                print('\n')
            print(self.puzzle[i],end=" ")
    #Return true when p is same as current state
    def isSamePuzzle(self,p):
        samePuzzle=True
        for i in range(len(p)):
            if self.puzzle[i]!=p[i]:
                samePuzzle=False
        return samePuzzle
    #Explore current node
    def expandNode(self):
        #get position of blank tile
        for i in range(len(self.puzzle)):
            if self.puzzle[i]==0:
                self.x=i
        #make move for them
        self.moveToRight(self.puzzle,self.x)
        self.moveToLeft(self.puzzle,self.x)
        self.moveToUp(self.puzzle,self.x)
        self.moveToDown(self.puzzle,self.x)
    #Copy contents from b to a    
    def copyPuzzle(self,a,b):
        for i in range(len(b)):
            a.append(b[i])

In [2]:
#For hashing
from collections import OrderedDict 
dict=OrderedDict
#Get all nodes in path from n to root node
def pathTrace(path,n):
        current=n
        path.append(current)
        while(current.parent!=None):
            current=current.parent
            path.append(current)
#Class to perform uniform search
class uninformedSearch:
    #Hashing function
    def hashing(self,x):
        hash_val=0
        # simply sum of index*state
        for i in range(len(x)):
            hash_val+=(i+1)**x[i]
        return hash_val
    #Perform uniform search
    def bfs(self,root):
        #No of explored nodes
        explored=0
        #No of children faced
        child=0
        #No of children put in memory at start root is in memory
        childm=1
        #Maximum memory needed for program
        maxm=childm
        #Stored solution path here
        pathToSolution=[]
        #2 hash tables (Dictionary)
        #To explore
        openList=dict()
        #Explored
        closedList=dict()
        #Adding root to to explore list
        openList[self.hashing(root.puzzle)]=root
        #Falg if goal found
        goalFound=False
        #Check if current is goal state and print it
        if root.goalTest():
            print("\nGoal Found")
            goalFound=True
            #Get path from current node(Goal) to source
            pathTrace(pathToSolution,root)
            return pathToSolution,explored,child,childm
        #Stop when there is nothing to explore or goal is found
        while len(openList)!=0 and goalFound==False:  
            #Get one node from list which contain node to explore
            _,currentNode=openList.popitem(last=False)
            #One node is out of memory
            childm-=1
            #Get hast value for that one node
            chash=self.hashing(currentNode.puzzle)
            #Mark node as explored as this node is going to be explored
            #It may have same hash also so check next
            if closedList.get(chash)!=None:
                tmp1=closedList.get(chash)
                while tmp1.next!=None:tmp1=tmp1.next
                tmp1.next=currentNode
            else:closedList[chash]=currentNode
            #Explore the node
            currentNode.expandNode()
            #Increament counter for explored
            explored+=1
            #For all childrens check goal state
            for i in range(len(currentNode.children)):
                #take ith child
                currentChild=currentNode.children[i]
                #Child is faced in process so increment counter
                child+=1
                #Check if this is goal state
                if currentChild.goalTest():
                    goalFound=True
                    #Get path to root
                    pathTrace(pathToSolution,currentChild)
                    break
                #Check if node in openList or closedList
                if(self.con(openList,currentChild,"open")==False and self.con(closedList,currentChild,"closed")==False):
                    #If not then just append it to open list
                    openList[self.hashing(currentChild.puzzle)]=currentChild
                    #So its added to open list so memory++
                    childm+=1
                    #Update maxm required
                    if childm>maxm:maxm=childm
        return pathToSolution,explored,child,maxm
        #pathToSolution: contain path from goal to root
        #explored: Number of nodes explored
        #child: Number of child faced
        #maxm: Maximum memory(Nodes) required to run code
    #Check if lis(HashTable) contains c(CurrentNode) with s(MODE)
    def con(self,lis,c,s):
        #Check if node is present in dictionary for hash value
        if lis.get(self.hashing(c.puzzle)) != None:
            #Check if node at hash value is same
            if lis[self.hashing(c.puzzle)].isSamePuzzle(c.puzzle):
                #Update cost to node if needed and in open list
                if s=="open" and lis[self.hashing(c.puzzle)].tmp>c.tmp:
                    lis[self.hashing(c.puzzle)].tmp=c.tmp
                    lis[self.hashing(c.puzzle)].parent=c.parent
                    lis[self.hashing(c.puzzle)].children=c.children
                #Node is alredy in list, so true
                return True
            #if same hash for 2 or more nodes
            else:
                #temporar pointer
                tmp1=lis[self.hashing(c.puzzle)]
                #Check all pointers
                while tmp1.next!=None:
                    tmp1=tmp1.next
                    if tmp1.isSamePuzzle(c.puzzle):
                        #Update cost to node if needed and in open list
                        if s=="open" and tmp1.tmp>c.tmp:
                            tmp1.tmp=c.tmp
                            tmp1.parent=c.parent
                            tmp1.children=c.children
                        #Node is alredy in list, so true
                        return True
                if(s=="open"):
                    tmp1.next=c
                    return True
        #False otherwise
        return False

In [3]:
#My randoly generated state
puzzle=[1,2,4,3,0,5,7,6,8]
#My goal state
goal=[0,1,2,3,4,5,6,7,8]
root=Node(puzzle)
ui=uninformedSearch()
#Call uniform search
solution,explored,child,childm=ui.bfs(root)
#Reverse it as solution is reverse(from goal to source)
solution.reverse()
#Print solution
if len(solution)>0:
    for i in range(len(solution)):
        print("\n\n\nMove",i)
        solution[i].printPuzzle()
        if i<len(solution)-1:
            print("\n\n******Next move**********")
    print("\n\n\nDepth of solution is : ",solution[-1].tmp)
    print("\nTotal moves required : ",len(solution)-1)
    print("\nTotal nodes explored : ",explored)
    print("\nTotal childrens : ",child)
    print("\nTotal children in memory at once : ",childm)
else:
    print("\nNo path to solution is found")




Move 0


1 2 4 

3 0 5 

7 6 8 

******Next move**********



Move 1


1 2 4 

3 6 5 

7 0 8 

******Next move**********



Move 2


1 2 4 

3 6 5 

7 8 0 

******Next move**********



Move 3


1 2 4 

3 6 0 

7 8 5 

******Next move**********



Move 4


1 2 0 

3 6 4 

7 8 5 

******Next move**********



Move 5


1 0 2 

3 6 4 

7 8 5 

******Next move**********



Move 6


0 1 2 

3 6 4 

7 8 5 

******Next move**********



Move 7


3 1 2 

0 6 4 

7 8 5 

******Next move**********



Move 8


3 1 2 

6 0 4 

7 8 5 

******Next move**********



Move 9


3 1 2 

6 4 0 

7 8 5 

******Next move**********



Move 10


3 1 2 

6 4 5 

7 8 0 

******Next move**********



Move 11


3 1 2 

6 4 5 

7 0 8 

******Next move**********



Move 12


3 1 2 

6 4 5 

0 7 8 

******Next move**********



Move 13


3 1 2 

0 4 5 

6 7 8 

******Next move**********



Move 14


0 1 2 

3 4 5 

6 7 8 


Depth of solution is :  14

Total moves required :  14

Total nodes explored :  3457

Tota

In [4]:
#Check for goal to goal
puzzle=[0,1,2,3,4,5,6,7,8]
goal=[0,1,2,3,4,5,6,7,8]
root=Node(puzzle)
ui=uninformedSearch()
solution,explored,child,childm=ui.bfs(root)
solution.reverse()
if len(solution)>0:
    for i in range(len(solution)):
        solution[i].printPuzzle()
        if i<len(solution)-1:
            print("\n\n******Next move**********",i)
else:
    print("\nNo path to solution is found")
print("\n\n\nTotal moves required : ",len(solution))
print("\nTotal nodes explored : ",explored)
print("\nTotal childrens : ",child)
print("\nTotal children in memory : ",childm)


Goal Found


0 1 2 

3 4 5 

6 7 8 


Total moves required :  1

Total nodes explored :  0

Total childrens :  0

Total children in memory :  1


In [5]:
#Same as before but return nodes at given depth
class configurations:
    #For hashing
    def hashing(self,x):
        hash_val=0
        for i in range(len(x)):
            hash_val+=(i+1)**x[i]
        return hash_val
    #To get nodes at depth depth
    def dbfs(self,root,depth):
        #root: Root node
        #depth: Depth at which to return nodes
        explored=0
        child=0
        childm=0
        maxm=0
        #This stores nodes which have required depth
        solutions=[]
        pathToSolution=[]
        openList=dict()
        closedList=dict()
        openList[self.hashing(root.puzzle)]=root
        goalFound=False
        #Check if current depth of root is same as required
        if root.tmp==depth:
            goalFound=True
            #Node is found, return path to it from source
            pathTrace(pathToSolution,root)
            solutions.append(pathToSolution)
            return solutions
        while len(openList)!=0 and goalFound==False:  
            _,currentNode=openList.popitem(last=False)
            chash=self.hashing(currentNode.puzzle)
            if closedList.get(chash)!=None:
                tmp1=closedList.get(chash)
                while tmp1.next!=None:tmp1=tmp1.next
                tmp1.next=currentNode
            else:closedList[chash]=currentNode
            currentNode.expandNode()
            explored+=1
            for i in range(len(currentNode.children)):
                currentChild=currentNode.children[i]
                #Skip nodes which are above depth
                if currentChild.tmp>depth:
                    goalFound=True
                    #Go out as we got what we needed
                    break
                child+=1
                if(self.con(openList,currentChild,"open")==False and self.con(closedList,currentChild,"closed")==False):
                    openList[self.hashing(currentChild.puzzle)]=currentChild
                    childm+=1
                    if childm>maxm:maxm=childm
                    #If node matches with depth then add it to solutions
                    if currentChild.tmp==depth:
                        pathTrace(pathToSolution,currentChild)
                        solutions.append(pathToSolution)
                        pathToSolution=[]

        return solutions
        #Check if lis(HashTable) contains c(CurrentNode) with s(MODE)
    def con(self,lis,c,s):
        #Check if node is present in dictionary for hash value
        if lis.get(self.hashing(c.puzzle)) != None:
            #Check if node at hash value is same
            if lis[self.hashing(c.puzzle)].isSamePuzzle(c.puzzle):
                #Update cost to node if needed and in open list
                if s=="open" and lis[self.hashing(c.puzzle)].tmp>c.tmp:
                    lis[self.hashing(c.puzzle)].tmp=c.tmp
                    lis[self.hashing(c.puzzle)].parent=c.parent
                    lis[self.hashing(c.puzzle)].children=c.children
                #Node is alredy in list, so true
                return True
            #if same hash for 2 or more nodes
            else:
                #temporar pointer
                tmp1=lis[self.hashing(c.puzzle)]
                #Check all pointers
                while tmp1.next!=None:
                    tmp1=tmp1.next
                    if tmp1.isSamePuzzle(c.puzzle):
                        #Update cost to node if needed and in open list
                        if s=="open" and tmp1.tmp>c.tmp:
                            tmp1.tmp=c.tmp
                            tmp1.parent=c.parent
                            tmp1.children=c.children
                        #Node is alredy in list, so true
                        return True
                if(s=="open"):
                    tmp1.next=c
                    return True
        #False otherwise
        return False

In [6]:
puzzle=[0,1,2,3,4,5,6,7,8]
#goal=[0,1,2,3,4,5,6,7,8]
d=8
show=10
root=Node(puzzle)
ui=configurations()
solutions=ui.dbfs(root,d)
print("Found",len(solutions),"solutions or configurations with depth",d,"\nShowing top :",show)
for i in range(len(solutions)):
    if len(solutions[i])>0:
        print("\n\nSolution",i+1,"with depth",d)
        solutions[i][0].printPuzzle()
    if i<len(solutions)-1 and i<show-1:
            print("\n\n******Next solution**********")
    if i>=show-1:break

Found 114 solutions or configurations with depth 8 
Showing top : 10


Solution 1 with depth 8


2 5 0 

1 3 4 

6 7 8 

******Next solution**********


Solution 2 with depth 8


2 3 5 

1 0 4 

6 7 8 

******Next solution**********


Solution 3 with depth 8


1 2 5 

6 3 4 

7 8 0 

******Next solution**********


Solution 4 with depth 8


1 2 5 

6 0 4 

7 3 8 

******Next solution**********


Solution 5 with depth 8


1 5 4 

3 2 8 

6 7 0 

******Next solution**********


Solution 6 with depth 8


3 1 5 

2 0 4 

6 7 8 

******Next solution**********


Solution 7 with depth 8


3 1 5 

6 2 4 

0 7 8 

******Next solution**********


Solution 8 with depth 8


1 2 5 

3 0 7 

6 8 4 

******Next solution**********


Solution 9 with depth 8


1 2 0 

3 7 5 

6 8 4 

******Next solution**********


Solution 10 with depth 8


1 2 5 

7 0 4 

3 6 8 

In [7]:
import time as t
import sys
node_size=sys.getsizeof(Node([0,1,2,3,4,5,6,7,8]))
#Tabular format
goal=[0,1,2,3,4,5,6,7,8]
print("Init state\t\t\tTotal time\t\t\tTotal children in memory")
#print("Init state\t\t\tAct Depth\tNodes explored\tTotal time\tTotal children in memory")
for i in solutions:
    tmp=t.time()
    puzzle=i[0].puzzle
    root=Node(puzzle)
    ui=uninformedSearch()
    solution,explored,child,childm=ui.bfs(root)
    if solution[0].tmp==d:
        print(solution[-1].puzzle,"\t",t.time()-tmp,"\t\t",childm*node_size)
#        print(solution[-1].puzzle,"\t",solution[0].tmp,"\t\t",explored,"\t\t",t.time()-tmp,"\t\t",childm*node_size)

Init state			Total time			Total children in memory
[2, 5, 0, 1, 3, 4, 6, 7, 8] 	 0.004433631896972656 		 4032
[2, 3, 5, 1, 0, 4, 6, 7, 8] 	 0.010878801345825195 		 7296
[1, 2, 5, 6, 3, 4, 7, 8, 0] 	 0.0068073272705078125 		 4032
[1, 2, 5, 6, 0, 4, 7, 3, 8] 	 0.011528968811035156 		 8128
[1, 5, 4, 3, 2, 8, 6, 7, 0] 	 0.007970571517944336 		 6336
[3, 1, 5, 2, 0, 4, 6, 7, 8] 	 0.008999347686767578 		 5952
[3, 1, 5, 6, 2, 4, 0, 7, 8] 	 0.007648468017578125 		 7104
[1, 2, 5, 3, 0, 7, 6, 8, 4] 	 0.006730556488037109 		 5440
[1, 2, 0, 3, 7, 5, 6, 8, 4] 	 0.0074672698974609375 		 7104
[1, 2, 5, 7, 0, 4, 3, 6, 8] 	 0.009298086166381836 		 6528
[1, 2, 5, 4, 0, 8, 3, 6, 7] 	 0.007270336151123047 		 6400
[1, 2, 0, 3, 8, 5, 6, 4, 7] 	 0.0066623687744140625 		 6656
[1, 2, 5, 3, 8, 7, 6, 4, 0] 	 0.008017301559448242 		 6592
[1, 2, 5, 6, 3, 8, 0, 4, 7] 	 0.006994009017944336 		 6720
[1, 5, 0, 3, 2, 8, 6, 4, 7] 	 0.006474018096923828 		 5568
[0, 1, 5, 3, 2, 8, 6, 4, 7] 	 0.007439613342285156 		 5504
[3

In [8]:
import time as t
import sys
node_size=sys.getsizeof(Node([0,1,2,3,4,5,6,7,8]))
#Tabular format
goal=[0,1,2,3,4,5,6,7,8]
print("\\begin{center}")
print("\\begin{tabular}{ |c|c|c| }")
print("Init state & Time(msec) & Memory\\\\")
count=0
#Solutions is list of instances of 8-puzzle
for i in solutions:
    count+=1
    tmp=t.time()
    puzzle=i[0].puzzle
    root=Node(puzzle)
    ui=uninformedSearch()
    solution,explored,child,childm=ui.bfs(root)
    #to devide in half
    if count==35:
        print("\\end{tabular}")
        print("\\end{center}") 
        print("\\begin{center}")
        print("\\begin{tabular}{ |c|c|c| }")
        print("Init state & Time(msec) & Memory\\\\")
    if solution[0].tmp==d:
        print("text{",solution[-1].puzzle,"} & ",int((t.time()-tmp)*1000000)," & ",childm*node_size,end="\\\\\n")
print("\\end{tabular}")
print("\\end{center}") 

\begin{center}
\begin{tabular}{ |c|c|c| }
Init state & Time(msec) & Memory\\
text{ [2, 5, 0, 1, 3, 4, 6, 7, 8] } &  4251  &  4032\\
text{ [2, 3, 5, 1, 0, 4, 6, 7, 8] } &  9333  &  7296\\
text{ [1, 2, 5, 6, 3, 4, 7, 8, 0] } &  4187  &  4032\\
text{ [1, 2, 5, 6, 0, 4, 7, 3, 8] } &  9128  &  8128\\
text{ [1, 5, 4, 3, 2, 8, 6, 7, 0] } &  8860  &  6336\\
text{ [3, 1, 5, 2, 0, 4, 6, 7, 8] } &  6997  &  5952\\
text{ [3, 1, 5, 6, 2, 4, 0, 7, 8] } &  7184  &  7104\\
text{ [1, 2, 5, 3, 0, 7, 6, 8, 4] } &  7860  &  5440\\
text{ [1, 2, 0, 3, 7, 5, 6, 8, 4] } &  8868  &  7104\\
text{ [1, 2, 5, 7, 0, 4, 3, 6, 8] } &  8433  &  6528\\
text{ [1, 2, 5, 4, 0, 8, 3, 6, 7] } &  9134  &  6400\\
text{ [1, 2, 0, 3, 8, 5, 6, 4, 7] } &  6680  &  6656\\
text{ [1, 2, 5, 3, 8, 7, 6, 4, 0] } &  7405  &  6592\\
text{ [1, 2, 5, 6, 3, 8, 0, 4, 7] } &  76196  &  6720\\
text{ [1, 5, 0, 3, 2, 8, 6, 4, 7] } &  5562  &  5568\\
text{ [0, 1, 5, 3, 2, 8, 6, 4, 7] } &  5658  &  5504\\
text{ [3, 1, 4, 5, 0, 2, 6, 7, 8] } &  787